1. Обучите рекуррентную нейронную сеть с одним из вариантов embedding
(например word2vec, GLOVE) . Продемонстрируйте на примере своего
собственного отзыва работу обученной вами нейронной сети.

In [2]:
!wget -O kinopoisk.zip https://drive.google.com/u/0/uc?id=1KXDwG6ccYXBM8ANV8hFL2HrQ4ypUCeij&export=download
!unzip kinopoisk.zip

!wget -O glove.6B.zip http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d glove.6B

--2023-04-27 07:06:51--  https://drive.google.com/u/0/uc?id=1KXDwG6ccYXBM8ANV8hFL2HrQ4ypUCeij
Resolving drive.google.com (drive.google.com)... 108.177.119.100, 108.177.119.113, 108.177.119.138, ...
Connecting to drive.google.com (drive.google.com)|108.177.119.100|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1KXDwG6ccYXBM8ANV8hFL2HrQ4ypUCeij [following]
--2023-04-27 07:06:51--  https://drive.google.com/uc?id=1KXDwG6ccYXBM8ANV8hFL2HrQ4ypUCeij
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qcf0fe29d3flljlmgkureu7i8v7p7e28/1682579175000/11248325512480947455/*/1KXDwG6ccYXBM8ANV8hFL2HrQ4ypUCeij?uuid=fc1cdde1-ce1e-4ee1-a4d9-e83ec24972cc [following]
--2023-04-27 07:06:55--  https://doc-0s-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qcf0fe

In [10]:
import os

TRAIN_DATA_DIR = '/content/dataset'

labels_index = {'pos':0, 'neu':1, 'neg':2}  # словарь сопоставления имени метки с числовым идентификатором

#Функция для загрузки тестовых данных.
def get_test_data():
  texts = []
  labels = []
  texts.append('Фильм посмотрел случайно - от скуки. Честно говоря можно было и не смотреть. Обычный, тупой американский боевик, который попытались сделать ещё и драмой.\
Фильм сводится к тому что одни негры жестоко убивают других, а Уиллис после Армагедона никак не может остановится спасать всех и вся.\
С другой стороны - отличное пособие для бойцов СПЕЦНАЗА. Вот что бывает когда нарушаешь дисциплину и не выполняешь прямого приказа - Потеря личного состава 60% и задание почти провалено!!!')
  labels.append(labels_index['neg'])
  texts.append('«Криминальное чтиво» — кинофильм культового режиссера Тарантино. Сюжет фильма нелинеен, как и почти во всех остальных работах Тарантино.\
Этот приём стал чрезвычайно популярен, породив множество подражаний во второй половине 1990-х. Название является отсылкой к популярным в середине ХХ века в США pulp-журналам.\
Именно в стиле таких журналов были оформлены афиши, а позднее саундтрек, видеокассеты.\
Фильм постоянно находится в первой десятке списка 250 лучших фильмов пользовательей IMDb. Обладатель премии «Оскар» за лучший сценарий, «Золотой пальмовой ветви» Каннского кинофестиваля 1994 года и ещё более сорока кинематографических наград.\
Картина также имела большой успех у публики и считается важной вехой в истории кинематографа, давшей ощутимый толчок развитию независимого американского кино. В 2013 году внесён в Национальный реестер фильмов США как имеющий культурное, историческое и эстетическое значение.\
Двое бандитов Винсент Вега и Джулс Винфилд проводят время в философских беседах в перерыве между разборками и «решением проблем» с должниками своего криминального босса Марселласа Уоллеса. Параллельно разворачивается три истории.\
Вершина карьеры выдающегося режиссера. Фильм, который по словам режиссера он не смог превзойти. Сверх-оригинальное повествование отличного сюжета, который подается не в хронологическом порядке, играет фильму только на пользу. Интересные и харизматичные персонажи, диалоги - ставшие культовыми и конечно же гениальный юмор.\
Рекомендую смотреть в переводе Гоблина.\
10 из 10')
  labels.append(labels_index['pos'])
  return texts, labels

   #Функция для загрузки данных из набора данных в блокнот.
def get_data(data_dir):
    texts = []  # список образцов текста
    
    labels = []  # список label ids
    for name in sorted(os.listdir(data_dir)):
        path = os.path.join(data_dir, name)
        if os.path.isdir(path):
            if name=='pos' or name=='neu' or name=='neg':
                label_id = labels_index[name]
                for fname in sorted(os.listdir(path)):
                        fpath = os.path.join(path, fname)
                        text = open(fpath,encoding='utf8').read()
                        texts.append(text)
                        labels.append(label_id)
    return texts, labels

train_texts, train_labels = get_data(TRAIN_DATA_DIR)

test_texts, test_labels = get_test_data()

print(train_texts[0])
print(train_labels[0])

Долго я пытался заставить себя поглядеть этот фильм, но смущала его продолжительность. Такая ко многому обязывает. Вот я собрался с силами и начал просмотр.

Уже первые крупные планы мини-дисков дали мне понять, что кино про далекое будущее, ведь не брать же в качестве «продакт-плейсмента» на такую футуристичную постановку дискеты 5.25...

Сцены бунтов разворачиваются аккурат на тротуарах, чтобы не задеть мерседес главного героя, колесящего туда-сюда... 

И вот уже Ральф Файнс дома придается «воспоминаниям» о бывшей подруге. Сразу рождается куча уместных и неуместных аналогий с ранее виденными шедеврами на тему потери героями своих близких. И пусть в этот момент еще не ясно, погибла ли Джульет или сбежала, уже понятно, что дальше герой либо станет на тропу войны, чтобы за нее мстить, либо попадет в водоворот событий, которые его приведут к объекту вожделения.

Так-так, кажется первое впечатление сложилось. Идем дальше.

Классно, что режиссер показала свое видение будущего, где всем буд

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import Constant

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000 
EMBEDDING_DIM = 100 
VALIDATION_SPLIT = 0.2

#Векторизация этих образцов текста в двумерный целочисленный тензор с помощью Keras Tokenizer
#Tokenizer тренируется только для обучающих данных и используется для токенизации как обучающих, так и тестовых данных.
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS) 
tokenizer.fit_on_texts(train_texts) 
train_sequences = tokenizer.texts_to_sequences(train_texts) #Converting text to a vector of word indexes 
test_sequences = tokenizer.texts_to_sequences(test_texts) 
word_index = tokenizer.word_index 
print('Found %s unique tokens.' % len(word_index))

Found 49510 unique tokens.


In [12]:
import numpy as np
from sklearn.model_selection import train_test_split

#Преобразование в последовательности для загрузки в нейронную сеть. Макс. посл. len равно 1000, как установлено ранее
# начальное заполнение 0, пока вектор не станет размером MAX_SEQUENCE_LENGTH
trainvalid_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
trainvalid_labels = to_categorical(np.asarray(train_labels), 3)
test_labels = to_categorical(np.asarray(test_labels), 3)

# разделить обучающие данные на обучающий набор и проверочный набор
#indices = np.arange(trainvalid_data.shape[0])
#np.random.shuffle(indices)
#trainvalid_data = trainvalid_data[indices]
#trainvalid_labels = trainvalid_labels[indices]
#num_validation_samples = int(VALIDATION_SPLIT * trainvalid_data.shape[0])
#x_train = trainvalid_data[:-num_validation_samples]
#y_train = trainvalid_labels[:-num_validation_samples]
#x_val = trainvalid_data[-num_validation_samples:]
#y_val = trainvalid_labels[-num_validation_samples:]
x_train, x_val, y_train, y_val = train_test_split(trainvalid_data, trainvalid_labels, train_size = 0.8, random_state = 1)
#Это данные, которые мы будем использовать для обучения RNN
print('Splitting the train data into train and valid is done')

Splitting the train data into train and valid is done


In [13]:
print('Preparing embedding matrix.')

# во-первых, построим маппинг embeddings_index 
# к их вектору embedding
embeddings_index = {}
with open(os.path.join('glove.6B', 'glove.6B.100d.txt'),encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors in Glove embeddings.' % len(embeddings_index))
#print(embeddings_index["google"])

# подготовить матрицу встраивания - строки - это слова из word_index, столбцы - это вложения этого слова из GLOVE.
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # слова, не найденные в embeddings_index, будут состоять из нулей.
        embedding_matrix[i] = embedding_vector

# загрузить эти предварительно обученные embedding в слой Embedding
# обратите внимание, что мы устанавливаем trainable = False, чтобы embedding оставались фиксированными
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
print("Preparing of embedding matrix is done")

Preparing embedding matrix.
Found 400000 word vectors in Glove embeddings.
Preparing of embedding matrix is done


In [14]:
print("Defining and training an LSTM model, using pre-trained embedding layer")

rnnmodel = Sequential()
rnnmodel.add(embedding_layer)
rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(Dense(3, activation='sigmoid'))
rnnmodel.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Training the RNN')

rnnmodel.fit(x_train, y_train,
          batch_size=32,
          epochs=1,
          validation_data=(x_val, y_val))
score, acc = rnnmodel.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)
print (rnnmodel.summary())

Defining and training an LSTM model, using pre-trained embedding layer
Training the RNN
1/1 [==============================] - 0s 370ms/step - loss: 0.9629 - accuracy: 1.0000
Test accuracy with RNN: 1.0
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1000, 100)         2000100   
                                                                 
 lstm_2 (LSTM)               (None, 128)               117248    
                                                                 
 dense_2 (Dense)             (None, 3)                 387       
                                                                 
Total params: 2,117,735
Trainable params: 117,635
Non-trainable params: 2,000,100
_________________________________________________________________
None


In [32]:
prediction = rnnmodel.predict(test_data)

for y in prediction:
  print('Предсказанные категории: ', np.argmax( y, axis=None, out=None ))

for y in test_labels:
  print('Исходные категории:', np.argmax(y, axis=None, out=None))



1/1 [==============================] - 0s 232ms/step
Предсказанные категории:  2
Предсказанные категории:  0
Исходные категории: 2
Исходные категории: 0


2. Возьмите по 2 отзыва из каждой классификации и примените NER для
распознавания сущностей из текстов

In [39]:
!pip install --upgrade transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.3 MB/s eta 0:00:00


In [40]:
import spacy

from transformers import *

def get_test_data_NER():
  texts = []
  labels = []

  texts.append('Не совсем понимаю возникшего ажиотажа вокруг этого режиссёра, но его фильмы даже не скучны, а просто злят. Я до этого посмотрел его же Страх и ненависть в Лас-Вегасе и Король-рыбак — \
 такие же бесполезные траты времени, как и Обезьяны. До этого я считал, что самый худший фильм с Брюсом Уиллисом это Гудзонский ястреб, но неожиданно сделал открытие еще более отвратительное.\
Неприятная затянутость действий. Всюду грязь, бомжи и визуальный дискомфорт, причём поданный бездарно (идеальный вариант отвращения видел в «лепоте — замечательном фильме). Единственное, что порадовало — это концовка, причем не развязка.\
 Как фанат фантастики и пост-апокалиптики я полностью разочарован в фильме. Бред сумасшедшего, не более. \
3 из 10')
  labels.append(labels_index['neg'])

  texts.append('Купились на Тома Хэнкса и рождественскую тематику. Закручено, вроде, лихо: одинокий отец попадает в радиоэфир, ему приходят тысячи писем, но только одна твёрдо решила, что он точно будет её мужчиной. И неважно, что у неё свадьба через месяц.\
   Это судьба! На поверку оказывается, что всем всё пофигу, кроме шестилетнего паренька, сына Тома Хэнкса, который организовал всё: и радиоэфир, и свидание будущей неразлучной парочки, перед этим сбагрив бестолковую пассию отца. Короче, вот он - реальный супермен.\
    И мне кажется, если бы дело с радио не выгорело, он нашёл бы другой выход из ситуации. Так что паренёк - брат по разуму своего протеже из Один дома.\
Можно посмотреть, если уж совсем всё кисло и вы очень любите Тома Хэнкса.\
5 из 10')
  labels.append(labels_index['neg'])

  texts.append('Посмотрев только ранние работы этого замечательного режиссёра, я уже поняла насколько он гениален! Его фильмы полны остроумными диалогами, шутками и иронией. А сам образ Вуди Аллена - обаятельного, неуклюжего интеллектуала,\
 меня покорил. ''Энни Холл'' - наверно, одна из лучших его картин. Фильм построен практически на одних диалогах, что не делает его скучным. Наоборот, в этом весь Аллен, копающийся в себе и рассуждающий на всевозможные темы. \
 Даже, наверное, неважно о чём он говорит, а важно как. Я могу слушать его болтовню часами. Так разглагольствовать может только единственный и неповторимый Вуди Аллен! Браво!\
10 из 10')
  labels.append(labels_index['neu'])

  texts.append('Фильм сказочный, неправдоподобный. Про современную Золушку. Не ту, что день и ночь работает на мачеху и злых сестёр в ожидании принца, а нынешнюю, которая и не помышляет о личном счастье, а изо всех сил карабкается по карьерной лестнице,\
   забыв даже о том, что она-женщина.\
Много в фильме нестыковок. Например, как это попавший в далёкое будущее человек так легко за один день ко всему привыкает, и даже сам готовит тосты?\
Но надо понимать, что это - всего лишь фантазия сценариста.\
Это мечта всех женщин, чтобы однажды к ним из ниоткуда явился принц на белом коне и сделал своей принцессой.\
Жаль, что всего лишь мечта…\
8 из 10.')
  labels.append(labels_index['neu'])

  texts.append(' ''Помни'' - это именно тот фильм, что сделал режиссёра культовым для масс. Многие вообще считают, что это первая картина Нолана. Это не так, но всё же фильм стоящий. Как и многие последующие картины,\
   ''Помни'' использует метод нелинейного повествования, способный водить зрителя за нос до самого конца хронометража. Но подлинное мастерство режиссёра-сценариста заключается в том, чтобы этот обман не был явным.\
    Зритель очень хочет быть ошеломлённым, оставаться в неведении, но так, чтобы это было незаметно. Кристофер Нолан в фильме ''Помни'' действительно смог абсолютно на все сто справиться с поставленной задачей. \
    Создать картину-головоломку, взрывающую мозг и немного восприятие мира - это, несомненно, заслуживает слова ''шедевр''. Хоть и не люблю им попусту разбрасываться. \
    ''Помни'' - это шедевр. Конкретно режиссёра и конкретно жанра, а не всех времён и народов, конечно. Смотреть обязательно.')
  labels.append(labels_index['pos'])
  texts.append('Потрясающий фильм, наделенный интригой, отличным сюжетом, динамикой, красивой картинкой.\
Завязка фильма хоть и выглядит обычной, но она интригует. Развивая обороты, зритель медленно, но неожиданно для себя попадает в ураганный разворот сюжета,\
 где понять кто есть кто не возможно, и принять чью то сторону тоже. Опять же интрига не покидает нас, а только усиливает свое воздействие на зрителя. Экшн, перестрелки, погони захватывают.\
  Тема отношений между людьми тоже задета. Фильм не читается за ранее на перед, к интересному и неожиданному концу придем неожиданно.\
Вообще данные фильм можно назвать фильмом, где ярко и грамотно сочетается все намешанное в нем. Нетривиальный сюжет, который к тому же здорово продуман,интрига, которая держит в напряжении и увлекает сильнее и сильнее.\
 Яркая актерская игра, замечательные спецэффекты, все это радует глаз. Качественный фильм!\
10 из 10')
  labels.append(labels_index['pos'])
  return texts, labels

def get_entities_html(text, ner_result, title=None):
  """Visualize NER with the help of SpaCy"""
  ents = []
  for ent in ner_result:
    e = {}
    # add the start and end positions of the entity
    e["start"] = ent["start"]
    e["end"] = ent["end"]
    # add the score if you want in the label
    # e["label"] = f"{ent["entity"]}-{ent['score']:.2f}"
    e["label"] = ent["entity"]
    if ents and -1 <= ent["start"] - ents[-1]["end"] <= 1 and ents[-1]["label"] == e["label"]:
      # if the current entity is shared with previous entity
      # simply extend the entity end position instead of adding a new one
      ents[-1]["end"] = e["end"]
      continue
    ents.append(e)
  # construct data required for displacy.render() method
  render_data = [
    {
      "text": text,
      "ents": ents,
      "title": title,
    }
  ]
  spacy.displacy.render(render_data, style="ent", manual=True, jupyter=True)

test_texts, test_labels = get_test_data_NER()

#Применяем NER алгоритм
ner = pipeline("ner", model="xlm-roberta-large-finetuned-conll03-english")

for doc in test_texts:
  doc_ner = ner(doc)
  get_entities_html(doc, doc_ner)

/usr/local/lib/python3.9/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-large-finetuned-conll03-english/snapshots/33a83d9855a119c0453ce450858c07835a0bdbed/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-large-finetuned-conll03-english",
  "_num_labels": 8,
  "architectures": [
    "XLMRobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "B-LOC",
    "1": "B-MISC",
    "2": "B-ORG",
    "3": "I-LOC",
    "4": "I-MISC",
    "5": "I-ORG",
    "6": "I-PER",
    "7": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-LOC": 0,
    "B-MISC": 1,
    "B-ORG": 2,
    "I-LOC": 3,
    "I-MISC": 4,
    "I-ORG": 5,
    "I-PER": 6,
    "O": 7
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "mo

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-large-finetuned-conll03-english/snapshots/33a83d9855a119c0453ce450858c07835a0bdbed/pytorch_model.bin
All model checkpoint weights were used when initializing XLMRobertaForTokenClassification.

All the weights of XLMRobertaForTokenClassification were initialized from the model checkpoint at xlm-roberta-large-finetuned-conll03-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use XLMRobertaForTokenClassification for predictions without further training.
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-large-finetuned-conll03-english/snapshots/33a83d9855a119c0453ce450858c07835a0bdbed/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-large-finetuned-conll03-english",

loading file sentencepiece.bpe.model from cache at /root/.cache/huggingface/hub/models--xlm-roberta-large-finetuned-conll03-english/snapshots/33a83d9855a119c0453ce450858c07835a0bdbed/sentencepiece.bpe.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-large-finetuned-conll03-english/snapshots/33a83d9855a119c0453ce450858c07835a0bdbed/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-large-finetuned-conll03-english/snapshots/33a83d9855a119c0453ce450858c07835a0bdbed/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-large-finetuned-conll03-english",
  "_num_labels": 8,
  "architectures": [
    "XLMRobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "c